In [1]:
import urllib2
import numpy as np
import json
import pandas as pd
pd.set_option("display.max_columns",999)
from pandas.io.json import json_normalize
import os
import pickle

In [2]:
#number per page = 30 
total_records = 1000
per_page = 30
n=int(total_records/per_page)
alldata =[]    
#loop to get all pages (n) (API only allows a max of 100 enteries per page)
for i in xrange(0, n):
    try:
        print "Downloading Page {}".format(i)
        api ='http://api.inaturalist.org/v1/observations?page={}&quality_grade=any&identifications=any&captive=false&iconic_taxa%5B%5D=Animalia&place_id=14'.format(i)
        req = urllib2.Request(api)
        response = urllib2.urlopen(req)
        response_read = response.read()
        json_data=json.loads(response_read)
        df = json_normalize(json_data, 'results', meta = ['page','per_page','total_results'])
        alldata.append(df)
    except:
        pass  
        

In [3]:
pickle.dump(alldata, open("inat_pickle.p", "wb"))

In [4]:
iNat_df = pd.concat(alldata)

In [5]:
iNat_df.shape

(960, 68)

In [6]:
columns_to_del = ['annotations','comments','faves','flags','ofvs','outlinks','project_ids',
                  'project_ids_with_curator_id','project_ids_without_curator_id','project_observations',
                  'observation_photos',
                 'quality_metrics','sounds','tags','votes','non_owner_ids']
iNat_df.drop(columns_to_del,axis = 1, inplace=True)

In [ ]:
iNat_numpy= iNat_df.values

In [ ]:
iNat_df.head()

In [ ]:
iNat_df.info()

In [7]:
df.to_csv('iNat_data.csv')

In [8]:
import findspark
findspark.init()
import pyspark
sc = pyspark.SparkContext()

In [9]:
from pyspark.sql import SQLContext
sqlsc=SQLContext(sc)

In [10]:
sparkdf = sqlsc.createDataFrame(iNat_df)

In [11]:
sparkdf.printSchema()

root
 |-- cached_votes_total: long (nullable = true)
 |-- captive: boolean (nullable = true)
 |-- comments_count: long (nullable = true)
 |-- community_taxon_id: double (nullable = true)
 |-- created_at: string (nullable = true)
 |-- created_at_details: map (nullable = true)
 |    |-- key: string
 |    |-- value: long (valueContainsNull = true)
 |-- created_time_zone: string (nullable = true)
 |-- description: string (nullable = true)
 |-- faves_count: long (nullable = true)
 |-- geojson: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- geoprivacy: string (nullable = true)
 |-- id: long (nullable = true)
 |-- id_please: boolean (nullable = true)
 |-- identifications: array (nullable = true)
 |    |-- element: map (containsNull = true)
 |    |    |-- key: string
 |    |    |-- value: string (valueContainsNull = true)
 |-- identifications_count: long (nullable = true)
 |-- identifications_most_agree: boolean (nullable = true)
 |-- identi

In [ ]:
type(sparkdf.species_guess)

In [12]:
#sparkdf.groupby(data['species_guess']).max().show()
sparkdf.registerTempTable("results")


In [13]:
sqlsc.sql("select distinct quality_grade from results").show()

+-------------+
|quality_grade|
+-------------+
|     research|
|       casual|
|     needs_id|
+-------------+



In [ ]:
def flattenColumn(df, column_list):
    for column in column_list:
        flat_columns = df[column].apply(pd.Series)
        flat_columns.columns = [column+'_'+str(flat_column_name) for flat_column_name in list(flat_columns.columns)]
        df= df.drop(column, 1).merge(flat_columns, left_index=True, right_index=True)
    return df
column_list = ['created_at_details','geojson','identifications','observation_photos',
               'observed_on_details','photos', 'place_ids','taxon','user']
